### ref
- [ゼロから作るDeep Learning](https://www.amazon.co.jp/%E3%82%BC%E3%83%AD%E3%81%8B%E3%82%89%E4%BD%9C%E3%82%8BDeep-Learning-Python%E3%81%A7%E5%AD%A6%E3%81%B6%E3%83%87%E3%82%A3%E3%83%BC%E3%83%97%E3%83%A9%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%AE%E7%90%86%E8%AB%96%E3%81%A8%E5%AE%9F%E8%A3%85-%E6%96%8E%E8%97%A4-%E5%BA%B7%E6%AF%85/dp/4873117585/ref=sr_1_1?ie=UTF8&qid=1494484778&sr=8-1&keywords=0%E3%81%8B%E3%82%89%E4%BD%9C%E3%82%8B%E3%83%87%E3%82%A3%E3%83%BC%E3%83%97%E3%83%A9%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0)
- [pytorch.org](http://pytorch.org/)

# パーセプトロン・活性化関数・誤差関数・誤差逆伝播
## パーセプトロン
パーセプトロンはニューラルネットワークの元となったモデルで、入力と出力のみからなるモデルです。
このパーセプトロンはXORすら解けない程度でした。パーセプトロンの関数としての問題は、**線形**であったことです。
> ここでいう線形関数は、$f(x + y) = f(x) + f(y)$を満たすような関数です。

![linear](./images/linear.png)
![non-linear](./images/non_linear.png)

線形だと下のセルにある図1のような問題を解けません。図1にあるような問題を線形分離不可能と言います。
このような問題は複数の直線か曲線でないと分離できません。一方、
線形分離可能な問題というのはクラスタを直線で分離できるようなものです（fig 2.）。
![fig 1](./images/fig_01.png)
![fig 2](./images/fig_02.png)

## 活性化関数
ニューラルネットワークは非線形な関数ですが、シンプルには線形な関数を非線形にするために活性化関数を用います。
活性化関数を適用する位置はスライド参照。  
活性化関数はニューロンの発火にインスパイアされたもので、初めはステップ関数が使われていましたが、誤差を用いたパラメータ更新（学習）のことを
考えて、微分可能な`sigmoid`関数が用いられるようになりました。現在では`ReLU`やその派生が主流になっています。
`ReLU`が主流になっている理由としては、計算が速いことと微分が楽なことが挙げられます。  
ほとんどの問題で、「線形変換によってデータの次元数を減らし、活性化関数を適用する」と言う処理を繰り返して出力します。
![fig 3](./images/fig_03.png)
![fig 4](./images/fig_04.png)
![fig 5](./images/fig_05.png)
![fig 6](./images/fig_06.png)

## 誤差関数
誤差関数は各タスクに合わせて設計されます。回帰問題では二乗誤差、識別問題では交差エントロピー（sigmoid cross entropy、softmax cross entropy）
が主流です。  
モデルの出力を$\hat{\boldsymbol{y}}$、正解を$\boldsymbol{y}$とすると、
二乗誤差は、$|| \boldsymbol{y} - \hat{\boldsymbol{y}}||^2$、交差エントロピーは$ - \boldsymbol{y}\cdot \log \hat{\boldsymbol{y}}$
で計算できます。
表記に関しては誤差の値自体は$\bf{E}$、誤差関数は$L(\hat{\boldsymbol{y}}, \boldsymbol{y}), \ L$と表すことが多いです。

![fig 7](./images/fig_07.png)

普通、誤差は小さいほど良いです。　ただし、前回（4/16）に伝えた通り誤差は2種類あります。前回は図をお見せすることが
できませんでしたが、下の図[^1]（青が訓練誤差、オレンジが汎化誤差を表す）では実際に初めは2つの誤差が
順調に下がっていますが10epochあたりを境に汎化誤差は上がり始めています。そのため、このモデルは100epoch学習したもの
よりも10epochあたりのものの方が未知のデータに対して精度が高いと想像できます。

![overfitting](https://github.com/mitmul/chainer-handson/blob/master/DeepCNN_cifar10_result/loss.png?raw=true)

[^1]: image from https://github.com/mitmul/chainer-handson

### 誤差の計算について
**訓練誤差、汎化誤差をどういうときに計算し、どのように言葉を使い分けるかについて。**
訓練誤差と汎化誤差の計算方法は全く同じですがデータが*学習データかテストデータか*の違いがあります。
また、誤差がパラメータの更新に使われるかどうかという大きな違いもあります。
訓練誤差はパラメータ更新に使われますが、汎化誤差は学習の状況（過学習しているかなど）を見るためだけに計算します。
そのため、多くのライブラリでモデルが学習用の計算かどうかをフラグで管理したり、変数が勾配を保持しないようにするためのフラグがあります。
今回使うpytorchでは、モデルに対しては学習時には`model.train()`、評価（汎化誤差を計算する時や実際に運用する時）の時には`model.eval()`という
メソッドが、変数クラス`Variable`には変数定義時に`Variable(data, volatile=True)`などと宣言します。

## 誤差逆伝播
偏微分・連鎖律がベースになっています。実際に扱う場合はフレームワークがよしなにやってくれているので特に説明しません。
この誤差逆伝播で各レイヤーに誤差を渡し、その誤差を用いてパラメータを更新します。

ここでは、PyTorchを用いて（toy dataで）誤差逆伝播をほぼ意識することなく実行できることを
以下のコードを直下のセルにコピペして確認しましょう。`ctrl + b -> cmd + p -> shift + return`

---

```back_prop_check.py
import torch
from torch.autograd import Variable

x = Variable(torch.ones(2, 2), requires_grad=True)
z = (x + 2) * (x + 2) * 3
print('z: {}'.format(z))
out = z.mean()
print('out: {}'.format(out))
out.backward()
print(x.grad)
```

---

上のsnippetでは
${\rm out} = \dfrac{1}{4}\sum_{i=1}^4 z_i =  \dfrac{1}{4}\sum_{i=1}^4 3 (x_i + 2)^2$より
$\dfrac{\partial {\rm out}}{\partial x_i} = \dfrac{\partial {\rm out}}{\partial z_i} \dfrac{\partial z_i}{\partial x_i} = \dfrac{3}{2}(x_i + 2) |_{x_i = 1} = 4.5$
が正解

誤差逆伝播の確認↓

# ニューラルネット
## 学習の流れ

### ミニバッチ
データセットの大きさを$N$とするとデータセットは教師あり学習の場合は$(\boldsymbol{x}_i, \boldsymbol{y}_i)_{i= 1}^N$と表せます。
バッチ処理はデータセットのすべてのデータに対して誤差を計算し、パラメータを更新する方法ですが、$N = 10,000$を超えることがほとんどの場合、
誤差を計算する回数に対してアップデートの回数が少なく、学習に時間がかかります。そのため、一般的には$n = 32$ぐらいのデータ（ミニバッチ）ごとに
アップデートを行い、プログラム中ではミニバッチを1つの変数のように扱います。
データの次元数が$d_{data}$の時、ミニバッチの形は$n \times d_{data}$になります。

よく、「学習は100epochで1epochは300iteration」などの言い方をしますが、
データセットの大きさを$N$、ミニバッチの大きさを$n$とすると、イテレーション数は$N / n$となり、これが1epochに該当します。

<!--
## 実装
コードを書いていきましょう。
-->

## データの扱い方
ミニバッチを1つの変数として扱うので学習ループ内での各変数の形は`[batch_size, *data_shape]`になります。

## モデルの構造
モデルはレイヤー（線形変換）と活性化関数（非線形）で作られていますが、どちらも関数であることに変わりはないです。しかし、
レイヤーは重みが更新されるので学習するにつれてその出力は変化します。`モデルの構造 1`では箱と箱を結ぶ線が活性化関数を、箱自体がレイヤーを表しています。また、`モデルの構造 2`では活性化関数が省略されています。  
多くのライブラリではレイヤーと活性化関数のファイルのディレクトリが分けられています。

###  レイヤー
- Linear / Dense / Full Connected
- Convolutional
- Recurrent (LSTM)
- Normalisation
    - (Dropout)
    - Batch Normalization
    
### 活性化関数
- tanh
- sigmoid
- relu
    - leaky relu
    - prelu
    - elu
    
## 多層パーセプトロン（Multi Layer Perceptron）
### どんなモデルか
多層パーセプトロンは`Linear`レイヤーのみからなるニューラルネットワークです。
![mlp](./images/mat.png)
各ユニットについての計算は上の図内にある通りですし、行列の大きさを（縦 $\times$　横）で表すと、下の図のように行列の大きさを変更することができます。
![matrix](./images/matrix.png)

この性質を利用して、`Linear`レイヤーの計算ではミニバッチの計算を一度に行うことができます（下図）。
`Linear`レイヤーを扱う場合、入力データの形は（ミニバッチサイズ、（データの形））となります。
ここで説明した計算は、一度に複数の同じ形のニューラルネットワークの計算を行っているようなイメージです。

![linear](./images/linear_layer.png)

---

ここでは、まず有名な手書き数字データセットのMNISTを用いて多層パーセプトロンの実験を行います。
多層パーセプトロン（スライド参照）を実装しましょう。データセットはPyTorchに関数が用意されているのでそこから落とします。

MNISTのデータは$0$から$9$の10クラス分類で各データは$28 \times 28$の画像を下の図のように平らにした$784$次元です。
![flatten_mnist](./images/flatten_image.png)
この$784$次元のデータをいくつかの`Linear`を用いて$10$次元にしましょう。
ここで使えるレイヤーは

- `Linear`: 第1引数：入力データの次元数、第2引数：出力データの次元数
- `BatchNorm1d`: 引数は次元数
- `Dropout`: 引数は変数

で、活性化関数は

- `sigmoid`
- `relu`
- `tanh`

などです。

`import`する必要があるのは、

```mnit_mlp.py
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
```

In [ ]:
%matplotlib inline
import os  # path
import datetime
import numpy as np  # 行列演算
import matplotlib.pyplot as plt  # グラフ
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
_cuda = False
kwargs = {'num_workers': 1, 'pin_memory': True} if _cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=50, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=False, transform=transforms.ToTensor()),
    batch_size=50, shuffle=True, **kwargs)

In [ ]:
# モデルの定義
class SimpleLinear(nn.Module):
    
    def __init__(self, in_dim=784, out_dim=10, hidden_dim=100):
        super(SimpleLinear, self).__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x):
        x = F.relu(self.fc1(x.view(-1, 784)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)
    
    def predict(self, x):
        return nn.Softmax(self.forward(x))

In [ ]:
class SimpleLinearDr(nn.Module):
    
    def __init__(self, in_dim=784, hidden_dim=256, out_dim=10):
        super(SimpleLinearDr, self).__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.dr1 = nn.Dropout()
        self.fc2 = nn.Linear(hidden_dim, out_dim)
    
    def forward(self, x):
        x = F.relu(self.dr1(self.fc1(x.view(-1, 784))))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)

In [ ]:
def train(epoch, log_interval=100):
    model.train()  # モデルがロスを用いて更新するように設定
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()  # モデルのレイヤーが前のエポックでの勾配履歴を保持しているので消去
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += loss.data[0]
        loss.backward()  # 誤差逆伝播
        optimizer.step()  # パラメータを更新
        if batch_idx % log_interval == 0:
            print('Training')
            print('epoch: {} [{}/{} ({:.0f}%)]\tloss:{:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),
                                                                   100. * batch_idx / len(train_loader), loss.data[0]))
    return train_loss / len(train_loader)

def test(epoch):
    model.eval()  #  更新しないように設定
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1]
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)
    print('Test')
    print('Average loss: {:.4f},\tAccuracy: {}/{} ({:.0f}%)'.format(test_loss, correct, len(test_loader.dataset),
                                                                   100. * correct / len(test_loader.dataset)))
    return test_loss, correct

In [ ]:
def run(n_epoch, train, test):
    train_loss = []
    test_loss = []
    test_accuracy = []
    start = datetime.datetime.now()
    print('Training started at {}'.format(start.strftime('%Y-%m-%d, %H:%M')))
    for epoch in range(1, n_epoch + 1):
        if epoch == 1:
            loop_start = datetime.datetime.now()
        train_loss.append(train(epoch))
        t_loss, t_accuracy = test(epoch)
        test_loss.append(t_loss)
        test_accuracy.append(t_accuracy)
        if epoch == 1:
            loop_end = datetime.datetime.now()
            print('one epoch takes {} sec'.format((loop_end - loop_start).total_seconds()))
    end = datetime.datetime.now()
    print('Training finished at {}'.format(end.strftime('%Y-%m-%d, %H:%M')))
    print('Duration: {}'.format((end - start).total_seconds()))
    return train_loss, test_loss, test_accuracy

In [ ]:
def plot_results(train_loss, test_loss, test_accuracy=None, save=False, save_dir='./images/result'):
    time_stamp = datetime.datetime.now().strftime('%Y/%m/%d, %H:%M')
    f_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')
    plt.plot(list(range(1, len(train_loss) + 1)), train_loss, label='train loss')
    plt.plot(list(range(1, len(test_loss) + 1)), test_loss, label='test loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.title('loss result at {}'.format(time_stamp))
    plt.legend()
    if save:
        tmp_save_dir = os.path.join(save_dir, 'loss')
        f_name = os.path.join(tmp_save_dir, '{}.png'.format(f_time))
        if not os.path.exists(tmp_save_dir):
            os.makedirs(tmp_save_dir)
        print('path to save loss: {}'.format(f_name))
        plt.savefig(f_name)
    plt.show()
    
    if test_accuracy is not None:
        plt.plot(list(range(1, len(test_accuracy) + 1)), list(np.asarray(test_accuracy) / 100.0), label='test accuracy')
        plt.xlabel('epoch')
        plt.ylabel('accuracy')
        plt.grid()
        plt.title('accuracy result at {}'.format(time_stamp))
        plt.legend()
        if save:
            tmp_save_dir = os.path.join(save_dir, 'accuracy')
            f_name = os.path.join(tmp_save_dir, '{}.png'.format(f_time))
            if not os.path.exists(tmp_save_dir):
                os.makedirs(tmp_save_dir)
            print('path to save accuracy: {}'.format(f_name))
            plt.savefig(f_name)
        plt.show()

### ミニマリズムなパーセプトロン

In [ ]:
model = SimpleLinear()
optimizer = optim.Adam(model.parameters())
train_loss, test_loss, test_accuracy = run(100, train, test)
plot_results(train_loss, test_loss, test_accuracy, save=True)

### optimizerの違い

In [ ]:
model = SimpleLinear()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optim_results = run(100, train, test)
plot_results(*optim_results)

### Dropoutを追加

In [ ]:
model = SimpleLinearDr()
optimizer = optim.Adam(model.parameters())
dr_results = run(100, train, test)
plot_results(*dr_results)

In [ ]:
class ComplexLinear(nn.Module):
    
    def __init__(self, in_size=784, hidden_1=256, hidden_2=128, hidden_3=64, out_size=10, dr=True):
        super(ComplexLinear, self).__init__()
        self.dr_flag = dr
        self.fc1 = nn.Linear(in_size, hidden_1)
        if dr:
            self.dr1 = nn.Dropout()
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        if dr:
            self.dr2 = nn.Dropout()
        self.fc3 = nn.Linear(hidden_2, hidden_3)
        if dr:
            self.dr3 = nn.Dropout()
        self.fc4 = nn.Linear(hidden_3, out_size)
    
    def forward(self, x):
        if self.dr_flag:
            h1 = F.relu(self.dr1(self.fc1(x.view(-1, 784))))
            h2 = F.relu(self.dr2(self.fc2(h1)))
            h3 = F.relu(self.dr3(self.fc3(h2)))
        else:
            h1 = F.relu(self.fc1(x.view(-1, 784)))
            h2 = F.relu(self.fc2(h1))
            h3 = F.relu(h2)
        h4 = self.fc4(h3)
        return F.log_softmax(h4)

In [ ]:
# complex model
model = ComplexLinear()
optimizer = optim.Adam(model.parameters())
complex_w_dr_results = run(100, train, test)

In [ ]:
plot_results(*complex_w_dr_results)

In [ ]:
model = SimpleLinearDr()
optimizer = optim.Adam(model.parameters())
results_dr = run(100, train, test)
plot_results(*results_dr)

# オートエンコーダー
前回、あんなにオートエンコーダーの話をしたのに、ここでしないわけにいきません。
オートエンコーダでは、データの大きさは砂時計のように入出力時が大きく、中間で一番小さくなるように設計します。ここでも、各層での次元数は任意です。今回はクラス数に合わせて、$10$次元に潰してみましょう

![autoencoder](./images/autoencoder.png)

In [ ]:
class AutoEncoder(nn.Module):
    
    def __init__(self, in_size=784, hidden_size=256, latent_size=10):
        super(AutoEncoder, self).__init__()
        self.fc1 = nn.Linear(in_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, latent_size)
        self.fc3 = nn.Linear(latent_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, in_size)
    
    def forward(self, x):
        h = F.relu(self.fc1(x.view(-1, 784)))
        h = F.relu(self.fc2(h))
        h = F.relu(self.fc3(h))
        return self.fc4(h)
    
    def infer(self, x):
        h = F.relu(self.fc1(x.view(-1, 784)))
        return self.fc2(h)

In [ ]:
class AutoEncoderDR(nn.Module):
    
    def __init__(self, in_size=784, hidden_size=256, latent_size=10):
        super(AutoEncoderDR, self).__init__()
        self.fc1 = nn.Linear(in_size, hidden_size)
        self.dr1 = nn.Dropout()
        self.fc2 = nn.Linear(hidden_size, latent_size)
        self.fc3 = nn.Linear(latent_size, hidden_size)
        self.dr3 = nn.Dropout()
        self.fc4 = nn.Linear(hidden_size, in_size)
    
    def forward(self, x):
        x = x.view(-1, 784)
        for i in range(1, 4):
            x = getattr(self, 'fc{}'.format(i))(x)
            if i != 2:
                x = getattr(self, 'dr{}'.format(i))(x)
            x = F.sigmoid(x)
        return self.fc4(x)

In [ ]:
def train_ae(epoch, ae, log_interval=100):
    ae.train()
    train_loss = .0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        output = ae(data)
        reconstruction_loss = (output - data) * (output - data)
        reconstruction_loss = reconstruction_loss.mean()
        train_loss += reconstruction_loss.data[0]
        reconstruction_loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Training')
            print('epoch: {} [{}/{} ({:.0f}%)]\tloss:{:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),
                                                                   100. * batch_idx / len(train_loader), reconstruction_loss.data[0]))
    return train_loss / len(train_loader)

def test_ae(epoch, ae):
    ae.eval()
    test_loss = .0
    for (data, _) in test_loader:
        data = Variable(data, volatile=True)
        output = ae(data)
        reconstruction_loss = (output - data) * (output - data)
        test_loss += reconstruction_loss.mean().data[0]
    test_loss /= len(test_loader)
    print('Test')
    print('Average loss: {:.4f}'.format(test_loss))
    return test_loss

In [ ]:
def run_ae(n_epoch, train_ae, test_ae):
    train_loss = []
    test_loss = []
    start_time = datetime.datetime.now()
    f_start = start_time.strftime('%H%M')
    print('Training AutoEncoder started at {}'.format(start_time.strftime('%Y/%m/%d, %H:%M')))
    for epoch in range(1, n_epoch + 1):
        if epoch == 1:
            start = datetime.datetime.now()
        train_loss.append(train_ae(epoch, ae))
        test_loss.append(test_ae(epoch, ae))
        if epoch == 1:
            end = datetime.datetime.now()
            print('# one loop takes {} sec'.format((end - start).total_seconds()))
    end_time = datetime.datetime.now()
    print('Training AutoEncoder finishied at {}'.format(end_time.strftime('%Y/%m/%d, %H:%M')))
    print('Duration: {} sec'.format((end_time - start_time).total_seconds()))
    return train_loss, test_loss

In [ ]:
ae = AutoEncoder()
optimizer = optim.Adam(ae.parameters())
results_ae = run_ae(50, train_ae, test_ae)

In [ ]:
ae_bn = AutoEncoderBN()
optimizer = optim.Adam(ae.parameters())
results_ae = run_ae(100, train_ae, test_ae)

In [ ]:
ae_dr = AutoEncoderDR()
optimizer = optim.Adam(ae.parameters())
results_ae = run_ae(10, train_ae, test_ae)

In [ ]:
def calculate_latent(ae):
    ae.eval()
    x0_list = []
    x1_list = []
    label_list = [] 
    for (data, target) in test_loader:
        data = Variable(data, volatile=True)
        latent = ae.infer(data).data.numpy().transpose(1, 0)
        x0_list.extend(list(latent[0, :]))
        x1_list.extend(list(latent[0, :]))
        label_list.extend(list(target))
    return pd.DataFrame({'x0': x0_list, 'x1': x1_list, 'digit': label_list})

In [ ]:
def reconstruct_mnist(ae):
    ae.eval()
    original_data = []
    reconstruncted = []
    digit_list = []
    for (data, target) in test_loader:
        data = Variable(data, volatile=True)
        reconstruncted_data = np.transpose(ae(data).view(-1, 1, 28, 28).data.numpy(), (0, 2, 3, 1))
        original_data.extend(list(data.data.numpy().transpose(0, 2, 3, 1)))
        reconstruncted.extend(list(reconstruncted_data))
        digit_list.extend(list(target))
    return original_data, reconstruncted, digit_list

In [ ]:
print(type(result_ae))
print(type(result_ae[0]))
print(type(result_ae[0][0]))
print(result_ae[0][0].shape)

plt.imshow(result_ae[0][0][:, :, 0], cmap='gray')
plt.show()

plt.imshow(result_ae[1][0][:, :, 0], cmap='gray')